### Export data to CSV if needed

In [ ]:
#farmland_footprints.to_csv(r'C:\Users\daniel.stankiewicz\Desktop\OSM_PVfarmPotential-master\CSV files\farmland_footprints.csv')

### Import necessary modules

In [ ]:
import osmnx as ox
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely
from maxrect import get_maximal_rectangle
from shapely.geometry.multipolygon import MultiPolygon
from shapely.geometry.polygon import Polygon

### Define all helper functions

In [ ]:
def download_footprints(west, south, east, north,footprints_type = 'landuse'):
    footprints_gdf = ox.footprints_from_polygon(poly,footprint_type=footprints_type)
    
    return footprints_gdf


def filter_footprints(footprints_gdf, footprints_type = 'landuse', chosenFootprints = ['farmland','forest','grass','meadow']):
    filtered_gdf = footprints_gdf.loc[footprints_gdf[footprints_type].isin(chosenFootprints)]
    
    return filtered_gdf


def show_footprints(footprints_gdf):
    fig,ax = ox.plot_footprints(footprints_gdf)


### User input - bounding box required

In [ ]:
#west, south, east, north = 18.5685998807,50.8595154681,18.5807532856,50.8629457573
#west, south, east, north = 21.8400832534,53.5136264306,21.8566314104,53.518393078
#west, south, east, north = 21.890483,53.51505,21.910353,53.51971
west, south, east, north = 20.7724649696,52.2912388736,20.7842924401,52.2959529584

### Create polygon from bbox and download footprints for given region

In [ ]:
poly = ox.bbox_to_poly(north,south,east,west)
footprints_landuse_gdf = download_footprints(west,south,east,north)
filtered_footprints_landuse_gdf = filter_footprints(footprints_landuse_gdf)

In [ ]:
print("List of related footprints downloaded from given area:\n{0}\n\nList of related footprints after filtering:\n{1}".format(footprints_landuse_gdf.landuse.unique(),filtered_footprints_landuse_gdf.landuse.unique()))

### Plot each type of downloaded footprints

In [ ]:
forest_footprints = filtered_footprints_landuse_gdf[filtered_footprints_landuse_gdf['landuse']=='forest']
farmland_footprints = filtered_footprints_landuse_gdf[filtered_footprints_landuse_gdf['landuse']=='farmland']
meadow_footprints = filtered_footprints_landuse_gdf[filtered_footprints_landuse_gdf['landuse']=='meadow']
grass_footprints = filtered_footprints_landuse_gdf[filtered_footprints_landuse_gdf['landuse']=='grass']

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

# Plot forests
forest_footprints.plot(ax=ax, facecolor='green')

# Plot farmlands
farmland_footprints.plot(ax=ax, facecolor="blue")

# Plot meadows
meadow_footprints.plot(ax=ax, facecolor="black")

# Plot grass
grass_footprints.plot(ax=ax, facecolor='yellow')

plt.tight_layout()

### Make an intersection betweeen downloaded relation footprints and given boundry box

In [ ]:
landuse_bbox = filtered_footprints_landuse_gdf['geometry'].intersection(poly)
landuse_bbox_gdf = gpd.GeoDataFrame(geometry=landuse_bbox)
landuse_bbox_gdf = landuse_bbox_gdf[landuse_bbox_gdf.is_empty == False]
landuse_bbox_gdf.plot()

In [ ]:
print("landuse_bbox_gdf is in format {0} \nand landuse_bbox is in format {1}".format(type(landuse_bbox_gdf),type(landuse_bbox)))

### Create single polygon from intersected GeoDataFrame

In [ ]:
area_polygon = landuse_bbox_gdf.unary_union
area_polygon

In [ ]:
print("Intersected poly is type of:\n{0}".format(type(area_polygon)))

In [ ]:
list(area_polygon.exterior.coords)

In [ ]:
list(area_polygon.interiors)

In [ ]:
import numpy as np

In [ ]:
abc = np.asarray(area_polygon.exterior.coords)

In [ ]:
abc

In [ ]:
for i in area_polygon.exterior.coords:
    print(area_polygon.exterior.coords[i])

In [ ]:
area_polygon.exterior.coords[0]

In [ ]:
x,y = area_polygon.exterior.coords.xy

In [ ]:
coords = area_polygon.exterior.coords

In [ ]:
print(coords)

In [ ]:
coordinates1 = [ [x0, y0], [x1, y1], ... [xn, yn] ]

In [ ]:
coordinates_arrray = []

In [ ]:
coordinates = (x,y)

In [ ]:
zipped = zip(x, y)

In [ ]:
list(zipped)

In [ ]:
# get the maximally inscribed rectangle
ll, ur = get_maximal_rectangle(abc)

In [ ]:
poly_test = Polygon([x,y])

In [ ]:
ch.area

In [ ]:
ch.bounds

# Finalny obszar przekształcony na Polygon

#### Next step jak pozyskać koordynaty z Polygonu

#### Algorytm żeby znależć największy prostokątny obszar w danym zaznaczeniu i nanieść jego kontury na wykres